In [74]:
import pandas as pd
#import lxml.etree.ElementTree as ET
from lxml import etree
path = 'fdh-gallica/bpt6k205233j/alto/bpt6k205233j_038.xml'

In [147]:
tree = etree.parse(path)
root = tree.getroot()
full_name = ''
addr_found = False
complete_person = False
for elem in (root.findall(".//{http://bibnum.bnf.fr/ns/alto_prod}TextLine")):
    for line in elem.findall(".//{http://bibnum.bnf.fr/ns/alto_prod}String"):
        word = line.attrib['CONTENT']
        #print(word)
        if(complete_person):
            complete_person=False
            if(is_family_name(word)):
                print(full_name)
                full_name=''
                addr_found= False
                print()
        full_name += word
        full_name += ' '   
        if('.' in word and addr_found):
            complete_person = True
        if(is_address(word)):
            addr_found = True
        

ABZ AGIT ABZAC (C""1 Pauld'), 181, rue de La Pompe (xvi*). 

fACHER de MONTGASCON (B™ d') [et B"" née Jeanne de COURCY], 18, avenue d'Antin (vin"),– et de Villequier, ES è53 [Q], Caudebec-en-Caux (Seine- Inférieure) ^= Villequier. 

ACHERY de SAN" DONNINO (P" d'), Ex-camérier secret, 62, avenue de la Grande-Armée (xvn*) et eJj de Soisy-sous-Montmorency, El Ji fi [S] Soisy-sous-Montmorency (Seine-et-Oise). 

ACHON (B°n Renéd'), Capitaine au l"r chasseurs [et B™" née de LAMOLÈRE, C"– d'HUST], 124, rue de Provence (vin*), et ^5 de la Roche-de- Gennes, El Sf Gennes (Maine-et-Loire), gg [Or], Les Rosiers-sur-Loire. 

ACLOQUE (M™) née DUCHANOY, 26, avenue Mar- ceau (xvi*) – et j*5 de La Borde, Kl Saint-An- toiue-du-Rocher, j: (I au d N" S), âï Wettray (Indre-et-Loire). 

ACLOQUE (M™ A.), née SEBERT, 9, place des Etats- Unis (xvi1)- [3698.92]. 

ACLOQUE (Maurice), 19, rue de Presbourg (xvi1) et villa Monténégro [3°. 590], à Cannes (Alpes-Mari- times). 

AC-Y ACOLLAS (René), $(, Conseiller réf

In [144]:
addr = ['avenue', 'av.', 'rue', 'r.', 'boulevard', 'b.', 'mêmes adresses', 'même adresse', 'villa', 'place']
def is_address(str):
    return str in addr

In [130]:
def is_family_name(str):
    nb_up = 0
    for c in str:
        if(c.isupper()):
            nb_up += 1
    
    return (nb_up/len(str) > 0.6)

In [131]:
def nom_jeune_fille(str):
    if("née" not in str):
        return ""
    name = ''
    nee_found = False
    for word in str.split(" "):
        if(word == "née"):
            nee_found = True
        if(is_family_name(word) and nee_found):
            name = "née "+ word.upper()
            break
    
    return name
    

In [132]:
nom_jeune_fille("Arnaud CHAZEAU née GATeAU")

'née GATEAU'

***Test of the map***

In [1]:
import os
import pandas as pd
import folium

In [2]:
paris_coord = [48.864716, 2.349014]

In [10]:
m = folium.Map(location=paris_coord, zoom_start = 12)
m

In [11]:
notre_dame_coord = [48.852966, 2.349902]

In [12]:
folium.Marker(notre_dame_coord, popup="Notre-Dame de Paris").add_to(m)

In [13]:
m